<a href="https://colab.research.google.com/github/avocadopelvis/imdb-movie-reviews-sentiment-analysis/blob/main/sentiment_analysis_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
!mkdir ~/.kaggle

In [3]:
!cp kaggle.json ~/.kaggle/

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 66% 17.0M/25.7M [00:01<00:00, 11.2MB/s]
100% 25.7M/25.7M [00:01<00:00, 18.4MB/s]


In [6]:
!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [7]:
#load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud, STOPWORDS
# from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re, string, unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob, Word
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

### Load the dataset

In [8]:
imdb_data = pd.read_csv('/content/IMDB Dataset.csv')
print(imdb_data.shape)
imdb_data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


### Exploratory Data Analysis

In [9]:
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


### Sentiment Count

In [10]:
imdb_data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

##### We observe that there are equal number of negative and postive sentiment. Thus, the dataset is balanced.

### Splitting the dataset

In [11]:
#train dataset
train_reviews = imdb_data.review[:40000]
train_sentiments = imdb_data.review[:40000]

#test dataset
test_reviews = imdb_data.review[40000:]
test_sentiments = imdb_data.sentiment[40000:]

print(train_reviews.shape, train_sentiments.shape)
print(test_reviews.shape, test_sentiments.shape)

(40000,) (40000,)
(10000,) (10000,)


### Text Normalization

In [12]:
#Text Tokenization
tokenizer = ToktokTokenizer()

#Setting English Stopwords
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Cleaning the dataset

In [13]:
#removing the html strips
def strip_html(text):
  soup = BeautifulSoup(text, 'html.parser')
  return soup.get_text()

#removing the square brackets
def remove_square(text):
  return re.sub('\[[^]]*\]', '', text)

#removing special characters
def remove_special(text, remove_digits=True):
  pattern = r'[^a-zA-z0-9\s]'
  return re.sub(pattern, '', text)

#removing the noisy text
def remove_noise(text):
  text = strip_html(text)
  text = remove_square(text)
  text = remove_special(text)
  return text

imdb_data['review'] = imdb_data['review'].apply(remove_noise)

### Text Stemming

In [14]:
def simple_stemmer(text):
  ps = nltk.porter.PorterStemmer()
  text = ' '.join([ps.stem(word) for word in text.split()])
  return text

imdb_data['review'] = imdb_data['review'].apply(simple_stemmer)

### Removing stopwords

In [15]:
#set stopwords to english
stop = set(stopwords.words('english'))
print(stop)

{'s', 'your', "needn't", 'what', "shan't", 'between', 'those', 'wouldn', 'which', "should've", "mightn't", 'me', 'did', 'out', 'a', "shouldn't", 'same', 'when', 'she', 'shan', 'whom', 'after', 'mightn', 'we', 'until', 'just', "hadn't", 'than', "hasn't", 'do', 'am', 'don', 'yours', 'into', 'where', "wasn't", 'weren', 'of', 'mustn', 'himself', 'd', 'that', "you're", 'doing', 'below', 'shouldn', "doesn't", 'why', 'this', "that'll", 'only', 'too', 'have', 'such', 'has', 'in', 'was', 'once', 'with', 'before', 'be', 'does', 'their', 'if', 'how', 'most', 'but', "couldn't", 'to', 'down', 'her', 'an', 'while', 'no', 'and', 'having', "don't", 'nor', 'these', 'up', 'haven', "isn't", "you'd", 'is', 'during', 'its', "it's", 'few', 'needn', 'own', 'some', 'other', 'against', 'ma', 'll', 'as', 'off', 'themselves', 'or', 'my', 'who', 'couldn', "you'll", 'herself', 'here', 'ourselves', 'above', "haven't", 'myself', 'will', 'won', 'for', "you've", 'his', 'so', 'then', 'not', 'any', 'about', 'him', 'both

In [16]:
#removing the stopwords
def remove_stopwords(text, is_lower_case = False):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  if is_lower_case:
    filtered_tokens = [token for token in tokens if token not in stopword_list]
  else:
    filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text

imdb_data['review'] = imdb_data['review'].apply(remove_stopwords)

### Normalized Train Reviews

In [18]:
norm_train_reviews = imdb_data.review[:40000]
norm_train_reviews[0]

'one review ha mention watch 1 Oz episod youll hook right thi exactli happen meth first thing struck Oz wa brutal unflinch scene violenc set right word GO trust thi show faint heart timid thi show pull punch regard drug sex violenc hardcor classic use wordit call OZ nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda Em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare dodgi deal shadi agreement never far awayi would say main appeal show due fact goe show wouldnt dare forget pretti pictur paint mainstream audienc forget charm forget romanceoz doesnt mess around first episod ever saw struck nasti wa surreal couldnt say wa readi watch develop tast Oz got accustom high level graphic violenc violenc injustic crook guard wholl sold nickel inmat wholl kill order get away well manner middl class inmat turn prison bitch due lack street skill prison exp

### As shown above, we can clearly observed that the stopwords such as have been removed while the text have been stemmed as well.

### Normalized Test Reviews

In [19]:
norm_test_reviews = imdb_data.review[40000:]
norm_test_reviews[40001]

'wa excit see sitcom would hope repres indian candian found thi show funni produc cast probabl happi get bad good feed back becaus far concern get talk wa readi stereotyp problem becaus stereotyp exist reason usual true realli wasnt anyth funni stereotyp charact fresh boat dad doesnt understand hi daughter radic feminist muslim daughter way terribl actress young modern indian man tri run hi mosqu polit correct pretti good actor onli see get betterit veri contriv dialog doesnt flow well wa much potenti someth like thi sadli think fail dont realli care watch anoth episodei howev enjoy watch great canadian actress sheila mccarthi alway treat natur everyth doe bad daughter show doesnt act abil'

### Bag of Words Model

In [21]:
#count vectorizer
cv = CountVectorizer(min_df = 0, max_df = 1, binary = False, ngram_range = (1, 3))

#transformed train reviews
cv_train_reviews = cv.fit_transform(norm_train_reviews)
#transformed test reviews
cv_test_reviews = cv.transform(norm_test_reviews)

print('bow_cv_train:', cv_train_reviews.shape)
print('bow_cv_test:', cv_test_reviews.shape)

bow_cv_train: (40000, 6209089)
bow_cv_test: (10000, 6209089)


### Term Frequency-Inverse Document Frequenct Model 

In [24]:
#tfidf vectorizer
tv = TfidfVectorizer(min_df = 0, max_df = 1, use_idf = True, ngram_range = (1, 3))

#transformed train reviews
tv_train_reviews = tv.fit_transform(norm_train_reviews)
#tranformed test reviews
tv_test_reviews = tv.transform(norm_test_reviews)

print('tfidf_train:', tv_train_reviews.shape)
print('tfidf_test:', tv_test_reviews.shape)

tfidf_train: (40000, 6209089)
tfidf_test: (10000, 6209089)


### Labelling the sentiment text

In [27]:
lb = LabelBinarizer()

#transformed sentiment data
sentiment_data = lb.fit_transform(imdb_data['sentiment'])
print(sentiment_data.shape)

(50000, 1)


### Splitting the sentiment data

In [28]:
train_sentiments = sentiment_data[:40000]
test_sentiments = sentiment_data[40000:]

In [32]:
print(train_sentiments)

[[1]
 [1]
 [1]
 ...
 [1]
 [0]
 [0]]


In [30]:
print(test_sentiments)

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


### Modelling the dataset